# Huc 12 Predictors
## Test 19020302 HUC 8 incl chugach nforest
All Huc 12s- Intersect w/catchments and find catchment (find downstream most catchment) with most upstream contribution and build watersheds for all ds catchments

Keep -
mnwtd_slope
% lake
summer precip - Runs in R but need downstream point (centroid of ds catchment)
predict for huc8

Visualize by HUC12 - Join data back to Huc12


## Section
Import modules
Set/Create Workspaces
 * Create Huc12 Catchment dataset to store catchments that will have watersheds generated
 * Crate Catchment point dataset to store catchment centroids
Collect Data
 * Set up data dictionary with VPU as key and store path to source data
    * Slope - Can link to already created slope rasters from original covariates worflow
    * NHDPlus Waterbodies - Can link to waterbody feature clase from original covariate workflow
Merge Data
 * Merge data together and or copy to local/in_memory and check that all projections match



In [1]:
import os, arcpy, sys,datetime
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'sys paths {sys.path}')
print (datetime.datetime.now())


imports complete
sys paths ['C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\future-0.18.2-py3.7.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\pytz-2020.1-py3.7.egg', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

## Collect NHDPlus Datasets

In [3]:
#Create Dictionary to relate datasets to vpu
vpuDict = {}

#Path to folder with NHDPlus data separated by vpu
nhdPlusfol = r'D:\Basedata\NHDPlus'
rois = ['Cook_Inlet','Copper_River']
vpuList = []
regDict = {}

#NHD folders - Update to new data downloaded Nov 8, 2021
for roi in rois:
    arcpy.env.workspace = os.path.join(nhdPlusfol,roi)
    gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
    #print(gdbs)
    Cats = []
    VAA = []
    waterbodies = []
    flowlines = []

    for gdb in gdbs:
        arcpy.env.workspace = gdb
        huc = gdb[-20:-12]
        append_value(regDict,huc,roi)
        #print(huc)
        vpuList.append(huc)
        append_value(vpuDict,huc,gdb)
        datasets = arcpy.ListDatasets(feature_type='feature')
        for ds in datasets:
            for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                if fc == "NHDFlowline":
                    flowpath = os.path.join(arcpy.env.workspace, ds, fc)
                    flowname = 'flowline_' + str(huc)
                    append_value(vpuDict,huc,flowpath)
                #     print(f'Copying {flowname}...')
                #     flowcopypath = os.path.join(scratch, flowname)
                #     arcpy.management.CopyFeatures(flowpath, flowcopypath)
                #     flowlines.append(flowcopypath)
                elif fc == "NHDWaterbody":
                    waterbodyname = 'waterbody_' + str(huc)
                    #waterbodycopypath = os.path.join(scratch, waterbodyname)
                    waterpath = os.path.join(arcpy.env.workspace, ds, fc)
                    #print(f'Copying {waterbodyname}...')
                    append_value(vpuDict,huc,waterpath)
                    # arcpy.management.CopyFeatures(waterpath, waterbodycopypath)
                    # waterbodies.append(waterbodycopypath)
                elif fc == "NHDPlusCatchment":
                    catchpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, catchpath)
                elif fc == "WBDHU12":
                    hucpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, hucpath)

        vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
        #print(f'Vaa table at {vaapath} exists = {arcpy.Exists(vaapath)}')
        append_value(vpuDict,huc,vaapath)
        append_value(vpuDict,huc,roi)

vpuDict


{'19020202': ['D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\NHDPlus\\NHDPlusCatchment',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\WBD\\WBDHU12',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\Hydrography\\NHDFlowline',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\Hydrography\\NHDWaterbody',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\NHDPlusFlowlineVAA',
  'Cook_Inlet'],
 '19020301': ['D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\NHDPlus\\NHDPlusCatchment',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\WBD\\WBDHU12',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\Hydrography\\NHDFlowline',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb

## Collect TauDEM data and build input dictionaries
* Only focusing on Prince of Wales right now.

In [3]:
# #TauDEM data
# #Create Dictionary to relate datasets to vpu
# tauvpuDict = {}
#
# # Path to network folder with TauDEM outputs
# taufol = r"T:\\Aquatic\\AKSSF\\TauDEM"
#
# # Path to local folder with TauDEM outputs
# #taufol = r""
#
# # All rois
# #taurois = ['Bristol_Bay', 'Kodiak','Prince_William_Sound']
# taurois = ['Prince_William_Sound']
#
# # Lists and dictionaries to organize input datasets
# tauvpuList = []
# tauregDict = {}
#
# #NHD folders - Update to new data downloaded Nov 8, 2021
# for roi in taurois:
#     arcpy.env.workspace = os.path.join(taufol,roi)
#     taufols = arcpy.ListWorkspaces('*','Folder')
#     print(taufol)
#     tauCats = []
#     taustreams = []
#
#
#     for fol in taufols:
#         with arcpy.da.Walk(fol,)
#         arcpy.env.workspace = fol
#         tauvpuList.append(roi)
#         for fc in arcpy.ListFeatureClasses():
#             print(fc)


In [6]:
# Set data dir equal to directory containing the AKSSF regional sub-folders.
akssf_data_dir = r'D:\\GIS\\AKSSF'
nhd_data_dir = r'D:\\Basedata\\NHDPlus'

# Create dictionaries
inputDict = {"'":"",'"':""}
scrDict = {}

def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput


# Create scratch workspace
while True:
    try:
        userinput = replace_all((input('Input drive or directory to create scratch workspaces ex. \'W:\\GIS\\\'\n') or 'D:\\GIS\\'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            scratchdrive = userinput
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

huc12cv_scratchgdb = os.path.normpath(os.path.join(scratchdrive,r"huc12_Outputs\\huc12_scratch.gdb"))
huc12cv_scratchfol = os.path.dirname(huc12cv_scratchgdb)
print(f'Scratch folder will be created {huc12cv_scratchfol}\nScratch GDB at {huc12cv_scratchgdb} ')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile. \'"J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"\'\n') or
                                  'D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs'), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

print(f'Using AWC events from {awc_events}\n {"-"*100}')

# Create Scratch Workspaces and add to dictionary
if not arcpy.Exists(huc12cv_scratchfol):
    os.mkdir(huc12cv_scratchfol)
    print(f'Creating scratch folder {huc12cv_scratchfol}')
else:
    print(f'Scratch folder {huc12cv_scratchfol} already exists')
print('----------')

if not arcpy.Exists(huc12cv_scratchgdb):
    arcpy.CreateFileGDB_management(huc12cv_scratchfol,"huc12_scratch.gdb")
    print(f'Creating scratch gdb {huc12cv_scratchgdb}')
else:
    print(f'Scratch folder {huc12cv_scratchgdb} already exists')
print('----------')


# Create VPU output gdbs
for vpu in vpuList:
    vpu_name = os.path.basename(vpu)
    hucscratchgdbname = 'h12_'+ vpu_name + '.gdb'
    hucscratchgdbpath = os.path.join(huc12cv_scratchfol, hucscratchgdbname)
    append_value(scrDict,vpu_name,hucscratchgdbpath)
    if not arcpy.Exists(hucscratchgdbpath):
        arcpy.CreateFileGDB_management(huc12cv_scratchfol,hucscratchgdbname)
        print(f'Creating scratch gdb for vpu {vpu_name} at {hucscratchgdbpath}')
    else:
        print(f'Scratch gdb for {vpu_name} already created at {hucscratchgdbpath}')

    print('----------')
print(f'All scratch workspaces set')
scrDict



Scratch folder will be created W:\GIS\huc12_Outputs
Scratch GDB at W:\GIS\huc12_Outputs\huc12_scratch.gdb 
Path specified does not exist!
Please re-enter a valid path
Using AWC events from J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Creating scratch folder W:\GIS\huc12_Outputs
----------
Creating scratch gdb W:\GIS\huc12_Outputs\huc12_scratch.gdb
----------
Creating scratch gdb for vpu 19020202 at W:\GIS\huc12_Outputs\h12_19020202.gdb
----------
Creating scratch gdb for vpu 19020301 at W:\GIS\huc12_Outputs\h12_19020301.gdb
----------
Creating scratch gdb for vpu 19020302 at W:\GIS\huc12_Outputs\h12_19020302.gdb
----------
Creating scratch gdb for vpu 19020401 at W:\GIS\huc12_Outputs\h12_19020401.gdb
----------
Creating scratch gdb for vpu 19020402 at W:\GIS\huc12_Outputs\h12_19020402.gdb
----------
Creating scratch gdb 

{'19020202': 'W:\\GIS\\huc12_Outputs\\h12_19020202.gdb',
 '19020301': 'W:\\GIS\\huc12_Outputs\\h12_19020301.gdb',
 '19020302': 'W:\\GIS\\huc12_Outputs\\h12_19020302.gdb',
 '19020401': 'W:\\GIS\\huc12_Outputs\\h12_19020401.gdb',
 '19020402': 'W:\\GIS\\huc12_Outputs\\h12_19020402.gdb',
 '19020501': 'W:\\GIS\\huc12_Outputs\\h12_19020501.gdb',
 '19020502': 'W:\\GIS\\huc12_Outputs\\h12_19020502.gdb',
 '19020503': 'W:\\GIS\\huc12_Outputs\\h12_19020503.gdb',
 '19020504': 'W:\\GIS\\huc12_Outputs\\h12_19020504.gdb',
 '19020505': 'W:\\GIS\\huc12_Outputs\\h12_19020505.gdb',
 '19020601': 'W:\\GIS\\huc12_Outputs\\h12_19020601.gdb',
 '19020602': 'W:\\GIS\\huc12_Outputs\\h12_19020602.gdb',
 '19020800': 'W:\\GIS\\huc12_Outputs\\h12_19020800.gdb',
 '19020101': 'W:\\GIS\\huc12_Outputs\\h12_19020101.gdb',
 '19020102': 'W:\\GIS\\huc12_Outputs\\h12_19020102.gdb',
 '19020103': 'W:\\GIS\\huc12_Outputs\\h12_19020103.gdb',
 '19020104': 'W:\\GIS\\huc12_Outputs\\h12_19020104.gdb'}

## Section 2
### By Vector Processing Unit
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset


In [10]:
# List of HUC12s to be processed
import arcpy, operator, time, datetime

outletDict = {}
outletList = []
outletcats = []
outletcatpts = []

# Uncomment to run all
# proclist = vpuList

# Run subset
proclist = ['19020302']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for vpu in vpuList:
    if vpu in proclist:
        try:
            # Start vpu timing function
            vpu_start = time.time()
            print(('*'*100))
            print(f'Huc {vpu} will be processed')

            # Must match names because some items in dictionary are out of order
            for v in vpuDict[vpu]:
                # Inputs
                if 'NHDPlusCatchment' in v:
                    cats = v
                elif 'NHDFlowline' in v:
                    streams = v
                elif 'NHDWaterbody'in v:
                    lakes = v
                elif 'WBDHU12' in v:
                    hucs = v
                elif 'NHDPlusFlowlineVAA' in v:
                    vaas = v
                elif ':' not in v:
                    region = v
            print(f'INPUTS:\cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')
            #Output names and paths
            outletcatsname = 'HUC_' + str(vpu) +'_out_cats'
            outcatspath = os.path.join(scrDict[vpu],outletcatsname)
            outletcatptsname = 'HUC_' + str(vpu) +'_out_catspts'
            outcatptspath = os.path.join(scrDict[vpu],outletcatptsname)

            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['NHDPlusID','TotDASqKm']
            fields2 = fields + ['cat_ID_con']
            valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(vaas, fields)}
            hucselect = arcpy.SelectLayerByLocation_management(hucs,'INTERSECT',awc_events,'','NEW_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} of {arcpy.GetCount_management(hucs)} HUC12s in {vpu} intersect awc events input')
            print(('*'*100))
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            # for f in hucFields:
            #     print (f'{f.name} is of {f.type} type')
            vcount =1
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'{vcount}. Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {int(i):(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    append_value(outletDict, row[0], int(outcatch))
                    append_value(outletDict, row[0], vpu)
                    outletList.append(int(outcatch))
                    vcount+=1
                del(row)
            del(cur)

            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Huc {vpu} at {outcatspath}')
            print(('*'*100))

            # Copy outputs
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,scrDict[vpu],outletcatsname)
            arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')

            # Add total drainage km from value dict to feature classes and cat_ID_con from regDict
            upfcs = [outcatspath, outcatptspath]
            for upfc in upfcs:
                arcpy.AddField_management(upfc,fields[1],'TEXT')
                arcpy.AddField_management(upfc,fields2[2],'TEXT')
                with arcpy.da.UpdateCursor(upfc,fields2) as cur:
                    for row in cur:
                        row[1] = valueDict[row[0]][0]
                        row[2] = regDict[vpu] + '_' + str(int(row[0]))
                        cur.updateRow(row)
                    del(row)
                del(cur)

            outletcats.append(outcatspath)
            outletcatpts.append(outcatptspath)

            # End Vpu time
            vpu_stop = time.time()
            vpu_time = int (vpu_stop - vpu_start)
            print(f'{vpu} Elapsed time: ({datetime.timedelta(seconds=vpu_time)})')
            print(f'{"*"*60}')

        except Exception:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

    else:
        print(('-'*100))
        print(f'Huc {vpu} will not be processed')
        print(('-'*100),'\n')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')


----------------------------------------------------------------------------------------------------
Huc 19020202 will not be processed
---------------------------------------------------------------------------------------------------- 

----------------------------------------------------------------------------------------------------
Huc 19020301 will not be processed
---------------------------------------------------------------------------------------------------- 

****************************************************************************************************
Huc 19020302 will be processed
INPUTS:\cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Based

## Section 2.1
Calculate Distance to Coast from outlet catchment point to the nearest coastline as a straight line distance
 * Generate near table and export as seperate csv


In [23]:
import arcpy, datetime
import numpy as pd

# Input path to coastline
coast = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_NHD_Coastline"

# Merge all catchment outlet centroids together
outletsname = 'AKSSF_NHDPlus_awcHuc12_outlet_cats_points'
outletspath = os.path.join(hucscratchgdbpath, outletsname)
all_nhd_outlet_pts = arcpy.Merge_management(outletcatpts,outletspath)

# Start timing function
start = datetime.datetime.now()

print(f'Getting distance to coast {datetime.datetime.now()}...')
arcpy.analysis.Near(all_nhd_outlet_pts, coast, None, "NO_LOCATION", "NO_ANGLE", "GEODESIC", "NEAR_DIST NEAR_DIST")
arcpy.AlterField_management(all_nhd_outlet_pts,'NEAR_DIST','dist_catch_coast_km','dist_catch_coast_km' )

# Convert distance in meters to km
with arcpy.da.UpdateCursor(all_nhd_outlet_pts,['dist_catch_coast_km']) as cur:
    for row in cur:
        row[0] = row[0] * 0.001
        cur.updateRow(row)
    del(row)
del(cur)

stop = datetime.datetime.now()
elapsed  = stop - start
print (f'Process complete at {datetime.datetime.now()} time elapsed: {elapsed} ')

Getting distance to coast 2022-02-02 15:13:32.691017...
Process complete at 2022-02-02 15:13:58.877378 time elapsed: 0:00:26.186361 


### Convert to df and examine

In [25]:
import numpy
import pandas as pd

# Make catchment points df
cat_df = pd.DataFrame()
cat_field_list = []

for field in arcpy.ListFields(all_nhd_outlet_pts):
    cat_field_list.append(field.name)
cat_arr = arcpy.da.TableToNumPyArray(all_nhd_outlet_pts, ['cat_ID_con', 'NHDPlusID','dist_catch_coast_km','TotDASqKm'])
cat_df = pd.DataFrame(cat_arr)
cat_df = cat_df.set_index('cat_ID_con')
cat_df

,NHDPlusID,dist_catch_coast_km,TotDASqKm
cat_ID_con,,,
Cook_Inlet_75004400008879,7.500440e+13,0.129134,23.32322502
Cook_Inlet_75004400008242,7.500440e+13,33.035154,112.78277531
Cook_Inlet_75004400007224,7.500440e+13,0.018925,322.32710002
Cook_Inlet_75004400008539,7.500440e+13,23.968173,103.00622492
Cook_Inlet_75004400011230,7.500440e+13,16.691152,88.33480009
...,...,...,...
Cook_Inlet_75004400004344,7.500440e+13,6.468900,164.24760023
Cook_Inlet_75004400004213,7.500440e+13,14.543668,60.98154999
Cook_Inlet_75004400007351,7.500440e+13,12.646195,96.95405006


## Export csv

In [26]:
import os
#Export CSV to read into R
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
nhd_catpts_outname = 'AKSSF_NHDPlus_awcHuc12_19020302_dist_catch_coast_km.csv'
arcpy.da.NumPyArrayToTable(cat_arr,os.path.join(outdir,nhd_catpts_outname))
print('CSV export complete')
print('----------')

CSV export complete
----------


## Section 3
### Generate watersheds
Iterate over HUC12 catchment dataset and create watersheds
 * Append output wtd to wtd dataset - Link back to catchment using catID (create catIDcon for unique identifier field)

In [27]:
# Begin generating watersheds
# List of HUC12s to be processed
import arcpy, time, datetime, os
import pandas as pd
from functools import reduce

# steps 4-9 for loop to create watersheds
arcpy.env.workspace = huc12cv_scratchgdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

proclist = ['19020302']
# proclist = vpuList
wtdDict = {}

for vpu in vpuList:
    if vpu in proclist:
        try:
            # Start vpu timing function
            vpu_start = time.time()
            wtdList=[]
            print(f'Huc {vpu} will be processed')
            outgdb = scrDict[vpu]
            # Must match names because some items in dictionary are out of order
            for v in vpuDict[vpu]:
                # Inputs
                if 'NHDPlusCatchment' in v:
                    cats = v
                elif 'NHDFlowline' in v:
                    streams = v
                elif 'NHDWaterbody'in v:
                    lakes = v
                elif 'WBDHU12' in v:
                    hucs = v
                elif 'NHDPlusFlowlineVAA' in v:
                    vaas = v
                elif ':' not in v:
                    region = v
            print(vpu)
            print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')

            catscopy = arcpy.CopyFeatures_management(cats,os.path.join(outgdb,'huc_'+str(vpu)+'_cats'))
            # Get list of index names for cats merge and add index if not already created
            index_names = [i.name for i in arcpy.ListIndexes(catscopy)]

            if 'NHDPlusID_index' not in index_names:
                print (f'Creating index for {catscopy}')
                arcpy.AddIndex_management(catscopy,'NHDPlusID','NHDPlusID_index')
            else:
                print(f'{catscopy} Indexed')

            #watersheds feature dataset for storing fcs
            fdatname = 'Huc_' + str(vpu) + '_Watersheds'
            fdat = os.path.join(outgdb, fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {vpu}')

            vaa_df1 = pd.DataFrame(arcpy.da.TableToNumPyArray(vaas, ("NHDPlusID", "FromNode", "ToNode", "StartFlag")))
            stream_df = pd.DataFrame(arcpy.da.TableToNumPyArray(streams, ("NHDPlusID", "FType")))
            dfs = [vaa_df1, stream_df]
            vaa_df = reduce(lambda left,right: pd.merge(left,right,on='NHDPlusID',how="outer"), dfs)
            # remove pipelines
            vaa_df = vaa_df[(vaa_df['FType'] != 428 )]
            vaa_df

            c=1

            for id in outletList:
                # Start iter timing function
                iteration_start = time.time()

                print(f"{c}. Identifying watershed for: {str(id)} ({int(len(outletList))} - {c} Remain)")
                rec = [id]
                up_ids = []
                up_ids.append(rec)
                rec_len = len(rec)
                hws_sum = 0

                while rec_len != hws_sum:
                    fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
                    rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]
                    rec_len = len(rec)
                    rec_hws = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "StartFlag"]
                    hws_sum = sum(rec_hws)
                    up_ids.append(rec)
                #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer, where_clause=expression)
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,'wtd_'+ str(id))

                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(outwtd)
                append_value(wtdDict,vpu,outwtd)
                c=c+1

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(huc12cv_scratchgdb,'AKSSF_HUC_'+str(vpu)+'_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            arcpy.AddField_management(wtd_merge,'NHDPlusID','DOUBLE')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','NHDPlusID','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    # Pull nhdplus id from merge source and calculate fields
                    nhdplusid= int(row[0].split('\\')[-1:][0].split('_')[1])
                    row[1] = nhdplusid
                    row[2] = region + '_' + str(nhdplusid)
                    row[3] = nhdplusid
                    row[4] = str(nhdplusid)
                    cur.updateRow(row)
                del(row)
            del(cur)

            # Stop iteration timer
            vpu_stop = time.time()
            vpu_time = int (vpu_stop - vpu_start)
            print(f'{vpu} Elapsed time: ({datetime.timedelta(seconds=vpu_time)})')
            print(f'{"*"*60}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')


Huc 19020302 will be processed
19020302
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlusFlowlineVAA
region = Cook_Inlet

Creating index for W:\GIS\huc12_Outputs\h12_19020302.gdb\huc_19020302_cats
Starting watershed for: 75004400005344 -(1 of 99)
Elapsed time: (0:00:02)
************************************************************
Starting watershed for: 75004400011755 -(2 of 99)
Elapsed time: (0:00:02)
************************************************************
Starting watershed for: 75004400010352 -(3 of 99)
Elapsed time: (0:00:02)
************************************************************
Star

## Section 4
###Calculate covariates
Iterate over watersheds and calculate the following:
* Mean wtd slope
* % Lake Cover
* Mean LCLD
* % Glacier Cover
* % North

In [29]:
from arcpy.sa import *

# Path to lcld rasters
lcld_folder = r'D:\\Basedata\\LCLD_rasters_archive'

# Create Dictionary to link input raster/fc data from original covariate workflow
dataDict={'Cook_Inlet':[r"D:\\GIS\\AKSSF\\Cook_Inlet\\slope.tif",
                        r"D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\glaciers",
                        "D:\\GIS\\AKSSF\\Cook_Inlet\\north.tif"],
          'Copper_River':[r"D:\\GIS\\AKSSF\\Copper_River\\slope.tif",
                          r"D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\glaciers",
                          r"D:\\GIS\\AKSSF\\Copper_River\\north.tif"],
          'Prince_William_Sound':[r"D:\\GIS\\AKSSF\\Prince_William_Sound\\slope.tif",
                                  r"D:\\GIS\\AKSSF\\Prince_William_Sound\\Prince_William_Sound.gdb\\glaciers",
                                  r"D:\\GIS\\AKSSF\\Prince_William_Sound\\north.tif"]
          }
# Separate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Lists to store output tables
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_pernorth_taba_tables = []
wtd_slope_ztables = []
lcld_Ztables = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Iterate over merged watersheds and calculate covariates
arcpy.env.workspace = huc12cv_scratchgdb
for fc in arcpy.ListFeatureClasses('*wtds_merge'):
    vpu = fc[10:18]
    outgdb = scrDict[vpu]
    # Must match names because some items in dictionary are out of order
    for v in vpuDict[vpu]:
        # Inputs
        if 'NHDPlusCatchment' in v:
            cats = v
        elif 'NHDFlowline' in v:
            streams = v
        elif 'NHDWaterbody'in v:
            lakes = v
        elif 'WBDHU12' in v:
            hucs = v
        elif 'NHDPlusFlowlineVAA' in v:
            vaas = v
        elif ':' not in v:
            region = v
    print(vpu)
    print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')
    roi = region
    tableid = roi + '_' + str(vpu)
    print(tableid)
    # Set Slope raster from data dictionary
    slope_rast = dataDict[region][0]
    # Set glacier fc from data dictionary
    glac_fc = dataDict[region][1]
    # Set North raster from data dictionary
    north_rast = dataDict[region][2]

    wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
    if roi in nhdplus_dat:
        #lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb" - local
        lakes_fc = r"T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_NHDPlus_LakePond" # network copy
    # Set data and variables unique to regions with TauDEM Data
    elif roi in tauDem_dat:
        lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb"


    wtds = []

    # Slope variables
    wtd_merge_slope_table_name = tableid + "_Watershed_Merge_SlopeZstats"
    wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
    # Lakes Ponds variables
    wtd_merge_lp_table_name = tableid + "_Watershed_Merge_LakesPonds"
    wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
    # Glaciers
    wtd_merge_glac_table_name = tableid + "_Watershed_Merge_Glaciers"
    wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)
    # North facing
    wtd_merge_north_table_name = tableid + "_Watershed_Merge_North"
    wtd_merge_north_table_path = os.path.join(outgdb, wtd_merge_north_table_name)


    # Watershed slope Zonal Statistics
    print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
          f' region')

    # Slope Zonal statistics  for watersheds
    print(f'Calculating {roi} watershed slope zonal stats...')
    arcpy.env.snapRaster = slope_rast
    arcpy.env.cellSize = slope_rast
    wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                    zone_field = wtd_cur_fields[0],
                                                    in_value_raster = slope_rast,
                                                    out_table = wtd_merge_slope_table_path,
                                                    statistics_type='ALL'
                                                    )

    # Add region identifier field for watershed tables                                                )
    arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
    # Add cat_ID_Con field
    arcpy.AddField_management(wtd_slope_metrics_table,'cat_ID_con',field_type='TEXT')

    # Update region field
    with arcpy.da.UpdateCursor(wtd_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        for row in cur:
            row[0] = roi
            strval = str(row[1])
            row[2] = roi+"_"+strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)
    # Append watershed slope table to list
    wtd_slope_ztables.append(wtd_slope_metrics_table)

    # Percent Lakes Ponds using Tabulate Intersection for watersheds
    print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
    print('----------')
    wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                        zone_fields=wtd_cur_fields[0],
                                                        in_class_features=lakes_fc,
                                                        out_table=wtd_merge_lp_table_path,
                                                        class_fields='Ftype',
                                                        out_units="SQUARE_METERS"
                                                        )
    # Add region and cat id fields
    arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','wtd_lake_per','wtd_lake_per')
    arcpy.AlterField_management(wtd_lp_tabint,'AREA','wtd_lake_area_sqm','wtd_lake_area_sqm')
    arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
    arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
    arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
    wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
    print (wtdlpfields)
    with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
        for row in cur:
            strval = str(row[1])
            row[5] = roi
            row[6] = strval.replace('.0','')
            row[7] = roi +'_'+ strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)

    # Append watershed lakes ponds table to list
    wtd_lp_tabint_tables.append(wtd_lp_tabint)

    # Percent glaciers using Tabulate Intersection for watersheds
    print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
    print('----------')
    wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                        zone_fields=wtd_cur_fields[0],
                                                        in_class_features=glac_fc,
                                                        out_table=wtd_merge_glac_table_path,
                                                        class_fields='O1Region',
                                                        out_units="SQUARE_METERS"
                                                        )
    # Add region and cat id fields
    arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','wtd_glacier_per','wtd_glacier_per')
    arcpy.AlterField_management(wtd_glac_tabint,'AREA','wtd_glacier_area_sqm','wtd_glacier_area_sqm')
    arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
    arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
    arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
    wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
    print (wtdglacfields)
    with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
        for row in cur:
            strval = str(row[1])
            row[5] = roi
            row[6] = strval.replace('.0','')
            row[7] = roi +'_'+ strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)
    # Append watershed percent glacier table to list
    wtd_glac_tabint_tables.append(wtd_glac_tabint)

    # Tabulate Area with north grid and watersheds

    print(f'Begin tabulate area between {north_rast} and watersheds in {roi} region')
    print('----------')
    # Percent North Tabulate area for watersheds
    wtd_per_north_tabarea = arcpy.sa.TabulateArea(in_zone_data= wtd_merge,
                                                  zone_field= wtd_cur_fields[0],
                                                  in_class_data=north_rast,
                                                  class_field="Value",
                                                  out_table=wtd_merge_north_table_path
                                                  )
    # Add region and percent north fields
    arcpy.AlterField_management(wtd_per_north_tabarea,'CAT_ID_TXT','CAT_ID_TXT_DEL','CAT_ID_TXT_DEL')
    arcpy.AddField_management(wtd_per_north_tabarea, 'region', field_type='TEXT')
    arcpy.AddField_management(wtd_per_north_tabarea, 'wtd_north_per', field_type='Float')
    arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[0], field_type='TEXT')
    arcpy.AddField_management(wtd_per_north_tabarea, wtd_cur_fields[2], field_type='TEXT')
    wtdnorfields = [f.name for f in arcpy.ListFields(wtd_per_north_tabarea)]
    print (wtdnorfields)
    with arcpy.da.UpdateCursor(wtd_per_north_tabarea, wtdnorfields) as cur:
        for row in cur:
            strval = str(row[1])
            row[4] = roi
            row[5] = row[3]/(row[3]+row[2])*100
            row[6] = strval.replace('.0','')
            row[7] = roi +'_'+ strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)
    # Drop UPPERCASE field form tab area
    arcpy.DeleteField_management(wtd_per_north_tabarea,'CAT_ID_TXT_DEL')
    # Append watershed percent north table to list
    wtd_pernorth_taba_tables.append(wtd_per_north_tabarea)

    # Begin LCLD calculations
    walk = arcpy.da.Walk(lcld_folder, datatype='RasterDataset')
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            raspath = os.path.join(dirpath, filename)
            year = filename[0:4]
            lcld_outname = 'lcld_'+str(year)+'_zStats'
            lcld_outpath = os.path.join(outgdb, lcld_outname)
            print(f'Year: {year} - raster path {raspath}')
            colname = 'wtd_lcld_mn_' + str(year)
            # lcld zonal statistics as table for all akssf watersheds
            print(f'Calculating {filename} zonal stats for all AKSSF watersheds...')
            #arcpy.env.snapRaster = raspath
            #arcpy.env.cellSize = raspath
            try:
                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin zonal stats for {filename}')
                lcld_table = ZonalStatisticsAsTable(in_zone_data = fc,
                                                                zone_field = 'cat_ID_con',
                                                                in_value_raster = raspath,
                                                                out_table = lcld_outpath,
                                                                statistics_type='MEAN'
                                                                )
                # Append zTable to table list
                lcld_Ztables.append(lcld_outpath)
                arcpy.AlterField_management(lcld_table,'MEAN', colname,colname)
                proc_list = [row[0] for row in arcpy.da.SearchCursor(lcld_table,'cat_ID_con')]
                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {filename} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')


            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

19020302
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020302_HU8_GDB.gdb\NHDPlusFlowlineVAA
region = Cook_Inlet

Cook_Inlet_19020302
Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in Cook_Inlet region
Calculating Cook_Inlet watershed slope zonal stats...
Begin tabulate intersection between T:\\Aquatic\\AKSSF\\AKSSF_Hydrography.gdb\\AKSSF_NHDPlus_LakePond and watersheds in Cook_Inlet region
----------
['OBJECTID', 'cat_ID_txt', 'FType', 'wtd_lake_area_sqm', 'wtd_lake_per', 'region', 'cat_ID', 'cat_ID_con']
Begin tabulate intersection between D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\glac

## Examine tables and merge/export

In [30]:
dfs = []
for table in lcld_Ztables:
    tblname = table[-16:]
    print(tblname)
    dfname = tblname + '_arr'
    # Make df
    dfname = pd.DataFrame()
    lcld_field_list = []
    for field in arcpy.ListFields(table):
        lcld_field_list.append(field.name)
        #print(f'{field.name}')
    lcld_arr = arcpy.da.TableToNumPyArray(table, lcld_field_list)
    dfname = pd.DataFrame(lcld_arr)
    dfname = dfname.drop(['OBJECTID','ZONE_CODE', 'AREA', 'COUNT'],axis=1)
    dfname = dfname.set_index('cat_ID_con')
    dfs.append(dfname)

# Merge all data frames together
import numpy as np
from functools import reduce
lcld_df = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
lcld_df

lcld_2001_zStats
lcld_2002_zStats
lcld_2003_zStats
lcld_2004_zStats
lcld_2005_zStats
lcld_2006_zStats
lcld_2007_zStats
lcld_2008_zStats
lcld_2009_zStats
lcld_2010_zStats
lcld_2011_zStats
lcld_2012_zStats
lcld_2013_zStats
lcld_2014_zStats
lcld_2015_zStats
lcld_2016_zStats
lcld_2017_zStats
lcld_2018_zStats
lcld_2019_zStats


,wtd_lcld_mn_2001,wtd_lcld_mn_2002,wtd_lcld_mn_2003,wtd_lcld_mn_2004,wtd_lcld_mn_2005,wtd_lcld_mn_2006,wtd_lcld_mn_2007,wtd_lcld_mn_2008,wtd_lcld_mn_2009,wtd_lcld_mn_2010,wtd_lcld_mn_2011,wtd_lcld_mn_2012,wtd_lcld_mn_2013,wtd_lcld_mn_2014,wtd_lcld_mn_2015,wtd_lcld_mn_2016,wtd_lcld_mn_2017,wtd_lcld_mn_2018,wtd_lcld_mn_2019
cat_ID_con,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004400005344,520.509051,508.405433,505.377240,500.963714,500.532097,515.671108,509.995206,522.716878,505.710495,519.811718,507.799762,520.476156,520.574698,494.189799,497.637066,509.934038,510.466283,506.270304,503.402322
Cook_Inlet_75004400011755,564.444009,553.189850,546.676839,545.389134,543.951730,556.857700,552.409089,559.785748,551.837955,554.837759,550.503730,561.177930,553.761947,539.041351,542.309656,543.853671,548.696056,545.134762,540.182484
Cook_Inlet_75004400010352,509.551592,501.181799,491.832787,493.722468,494.759142,507.035132,498.694661,517.818537,491.611237,508.359131,500.773588,513.361570,511.525629,481.955786,481.891787,496.708530,496.161511,503.096208,497.693959
Cook_Inlet_75004400007996,504.175142,495.784529,489.857588,491.170970,489.220219,502.619685,495.572548,508.803808,490.665353,500.640127,495.863432,507.356903,509.346870,478.528016,484.090911,491.223736,490.668783,490.608826,493.742622
Cook_Inlet_75004400004850,484.997404,487.153397,451.456256,482.279436,469.999586,485.523631,481.105462,495.159942,478.876774,483.054496,483.135788,496.730896,502.512729,444.622105,451.619080,452.742992,479.077717,477.712043,475.224581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75004400010337,475.481973,482.757579,423.842961,477.607190,464.195834,473.934574,469.370365,473.545688,476.126278,474.148137,478.633391,479.490321,494.054570,466.436651,432.107032,441.154316,471.937621,475.286969,455.941913
Cook_Inlet_75004400008879,473.000779,479.593725,430.429066,472.965813,469.316728,477.245322,459.450837,476.794236,477.664237,470.348740,472.191278,481.256098,490.865165,469.645157,423.882979,438.889834,466.550128,468.182090,461.628005
Cook_Inlet_75004400004457,482.066856,485.844861,443.333986,484.460604,467.505422,485.081167,483.132374,491.201997,477.718749,481.870984,483.173043,496.106864,501.102991,438.825152,444.976096,451.657545,476.677755,476.303059,470.164198


In [31]:
# Export merged dataframe to csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
lcld_csv_out = os.path.join(outdir,'AKSSF_NHDPlus_awcHuc12_19020302_wtd_lcld_mn.csv')
lcld_df.to_csv(lcld_csv_out, encoding = 'utf-8')
print('Export dataframe to csv complete')

Export dataframe to csv complete


In [32]:
outgdb = huc12cv_scratchgdb

# Table names/paths
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_slope')
wtd_lcld_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_lcld')
wtd_per_north_table_out = os.path.join(outgdb, 'AKSSF_NHDPlus_awcHuc12_19020302_wtd_north_per')

# Merge all regional tables together
outtables = []
wtd_per_north = arcpy.Merge_management(wtd_pernorth_taba_tables, wtd_per_north_table_out)
arcpy.AlterField_management(wtd_per_north,"VALUE_0","non_north_area","non_north_area")
arcpy.AlterField_management(wtd_per_north,"VALUE_1","north_area","north_area")
outtables.append(wtd_per_north)
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')

Tables merged
----------


In [33]:
slopeDict = { 'ZONE_CODE': ('cat_slope_ZONE_CODE', 'wtd_slope_ZONE_CODE'),
         'COUNT': ('cat_slope_COUNT', 'wtd_slope_COUNT'),
          'AREA': ('cat_slope_AREA', 'wtd_slope_AREA'),
          'MIN': ('cat_slope_MIN', 'wtd_slope_MIN'),
          'MAX': ('cat_slope_MAX', 'wtd_slope_MAX'),
          'RANGE': ('cat_slope_RANGE', 'wtd_slope_RANGE'),
          'MEAN': ('cat_slope_MEAN', 'wtd_slope_MEAN'),
          'STD': ('cat_slope_STD', 'wtd_slope_STD'),
          'SUM': ('cat_slope_SUM', 'wtd_slope_SUM'),
          'VARIETY': ('cat_slope_VARIETY', 'wtd_slope_VARIETY'),
          'MAJORITY': ('cat_slope_MAJORITY', 'wtd_slope_MAJORITY'),
          'MINORITY': ('cat_slope_MINORITY', 'wtd_slope_MINORITY'),
          'MEDIAN': ('cat_slope_MEDIAN', 'wtd_slope_MEDIAN'),
          'PCT90': ('cat_slope_PCT90', 'wtd_slope_PCT90')
         }
# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)


ZONE_CODE wtd_slope_ZONE_CODE
COUNT wtd_slope_COUNT
AREA wtd_slope_AREA
MIN wtd_slope_MIN
MAX wtd_slope_MAX
RANGE wtd_slope_RANGE
MEAN wtd_slope_MEAN
STD wtd_slope_STD
SUM wtd_slope_SUM
MEDIAN wtd_slope_MEDIAN
PCT90 wtd_slope_PCT90


In [34]:
# Export copies of dbf tables as csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
for table in outtables:
    tablename = arcpy.Describe(table).basename + ".csv"
    tablepath = os.path.join(outdir,tablename)
    print(tablepath)
    arcpy.conversion.TableToTable(table, outdir, tablename)

C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_NHDPlus_awcHuc12_19020302_wtd_north_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_slope.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_glacier_per.csv
C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\AKSSF_NHDPlus_awcHuc12_19020302_huc12_wtd_lakepond_per.csv


In [35]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [37]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

,cat_ID_txt,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,wtd_slope_PCT90,region
cat_ID_con,,,,,,,,,,,,
Cook_Inlet_75004400005344,75004400005344,450061.00,45006100.00,0.00,70.87,70.87,25.38,13.03,11421329.52,26.37,41.61,Cook_Inlet
Cook_Inlet_75004400011755,75004400011755,1055899.00,105589900.00,0.00,76.85,76.85,24.02,15.07,25364061.16,24.16,44.20,Cook_Inlet
Cook_Inlet_75004400010352,75004400010352,535581.00,53558100.00,0.00,67.02,67.02,26.18,10.23,14019013.53,28.10,37.79,Cook_Inlet
Cook_Inlet_75004400007996,75004400007996,2825727.00,282572700.00,0.00,65.51,65.51,22.75,9.80,64285852.09,23.95,34.55,Cook_Inlet
Cook_Inlet_75004400004850,75004400004850,1563074.00,156307400.00,0.00,67.16,67.16,18.79,10.87,29374364.09,18.87,33.13,Cook_Inlet
...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75004400010337,75004400010337,344724.00,34472400.00,0.00,35.96,35.96,4.01,4.34,1380818.20,3.03,9.43,Cook_Inlet
Cook_Inlet_75004400008879,75004400008879,233265.00,23326500.00,0.00,30.82,30.82,1.95,2.25,455340.88,1.92,3.44,Cook_Inlet
Cook_Inlet_75004400004457,75004400004457,1026297.00,102629700.00,0.00,59.70,59.70,16.52,10.14,16950039.30,14.77,31.31,Cook_Inlet


In [38]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

,cat_ID_txt,FType,wtd_lake_area_sqm,wtd_lake_per,region,cat_ID
cat_ID_con,,,,,,
Cook_Inlet_75004400000141,75004400000141,390,1576148.40,3.57,Cook_Inlet,75004400000141
Cook_Inlet_75004400000153,75004400000153,390,58646461.98,11.14,Cook_Inlet,75004400000153
Cook_Inlet_75004400000171,75004400000171,390,8422188.80,7.60,Cook_Inlet,75004400000171
Cook_Inlet_75004400000334,75004400000334,390,104986117.90,4.68,Cook_Inlet,75004400000334
Cook_Inlet_75004400000370,75004400000370,390,7821176.38,1.61,Cook_Inlet,75004400000370
...,...,...,...,...,...,...
Cook_Inlet_75004400011230,75004400011230,390,161218.08,0.18,Cook_Inlet,75004400011230
Cook_Inlet_75004400011320,75004400011320,390,553571.25,0.40,Cook_Inlet,75004400011320
Cook_Inlet_75004400011322,75004400011322,390,17707.12,0.03,Cook_Inlet,75004400011322


In [39]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

,cat_ID_txt,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region,cat_ID
cat_ID_con,,,,,,
Cook_Inlet_75004400000334,75004400000334,1,193799125.72,8.64,Cook_Inlet,75004400000334
Cook_Inlet_75004400000370,75004400000370,1,291900930.05,60.11,Cook_Inlet,75004400000370
Cook_Inlet_75004400000627,75004400000627,1,221275.03,0.05,Cook_Inlet,75004400000627
Cook_Inlet_75004400000862,75004400000862,1,1216847.53,4.21,Cook_Inlet,75004400000862
Cook_Inlet_75004400001134,75004400001134,1,455445.69,0.98,Cook_Inlet,75004400001134
Cook_Inlet_75004400001136,75004400001136,1,12852242.35,8.40,Cook_Inlet,75004400001136
Cook_Inlet_75004400001464,75004400001464,1,3207932.70,4.28,Cook_Inlet,75004400001464
Cook_Inlet_75004400001569,75004400001569,1,528404086.60,11.56,Cook_Inlet,75004400001569
Cook_Inlet_75004400001616,75004400001616,1,42704030.84,7.99,Cook_Inlet,75004400001616


In [40]:
# Make watershed north df
wtd_n_df = pd.DataFrame()
wtd_n_field_list = []
for field in arcpy.ListFields(wtd_per_north):
    wtd_n_field_list.append(field.name)
wtd_n_arr = arcpy.da.TableToNumPyArray(wtd_per_north,wtd_n_field_list)
wtd_n_df = pd.DataFrame(wtd_n_arr)
wtd_n_df = wtd_n_df.drop("OBJECTID",axis=1)
wtd_n_df = wtd_n_df.set_index('cat_ID_con')
dfs.append(wtd_n_df)
wtd_n_df

,non_north_area,north_area,region,wtd_north_per,cat_ID_txt
cat_ID_con,,,,,
Cook_Inlet_75004400005344,38049800.00,6956300.00,Cook_Inlet,15.46,75004400005344
Cook_Inlet_75004400011755,77518800.00,28071100.00,Cook_Inlet,26.59,75004400011755
Cook_Inlet_75004400010352,42064400.00,11493700.00,Cook_Inlet,21.46,75004400010352
Cook_Inlet_75004400007996,208771000.00,73801700.00,Cook_Inlet,26.12,75004400007996
Cook_Inlet_75004400004850,117098200.00,39209200.00,Cook_Inlet,25.08,75004400004850
...,...,...,...,...,...
Cook_Inlet_75004400010337,28343000.00,6129400.00,Cook_Inlet,17.78,75004400010337
Cook_Inlet_75004400008879,19775200.00,3551300.00,Cook_Inlet,15.22,75004400008879
Cook_Inlet_75004400004457,72416200.00,30213500.00,Cook_Inlet,29.44,75004400004457


In [50]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)
df_final.columns = list(uniquify(df_final))

#List of final columns in the order to output
final_cols = ['cat_ID_txt', 'wtd_slope_COUNT', 'wtd_slope_AREA', 'wtd_slope_MIN', 'wtd_slope_MAX','wtd_slope_RANGE',
              'wtd_slope_MEAN', 'wtd_slope_STD', 'wtd_slope_SUM', 'wtd_slope_MEDIAN', 'wtd_slope_PCT90',
              'wtd_lake_area_sqm', 'wtd_lake_per', 'wtd_glacier_area_sqm', 'wtd_glacier_per','wtd_north_per' ]
#Create list of duplicate column names and drop
drop_cols = ['cat_ID_txt_y', 'region_y','FType', 'O1Region','cat_ID_y','cat_ID_x','non_north_area','north_area',
             'region_x_2', 'region_y_2', 'cat_ID_txt_y_2','cat_ID_txt_x_2']
df_final

,cat_ID_txt_x,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,...,O1Region,wtd_glacier_area_sqm,wtd_glacier_per,region_x_2,cat_ID_y,non_north_area,north_area,region_y_2,wtd_north_per,cat_ID_txt_y_2
cat_ID_con,,,,,,,,,,,,,,,,,,,,,
Cook_Inlet_75004400005344,75004400005344,450061.00,45006100.00,0.00,70.87,70.87,25.38,13.03,11421329.52,26.37,...,1,1588148.84,3.53,Cook_Inlet,75004400005344,38049800.00,6956300.00,Cook_Inlet,15.46,75004400005344
Cook_Inlet_75004400011755,75004400011755,1055899.00,105589900.00,0.00,76.85,76.85,24.02,15.07,25364061.16,24.16,...,1,47685961.13,45.16,Cook_Inlet,75004400011755,77518800.00,28071100.00,Cook_Inlet,26.59,75004400011755
Cook_Inlet_75004400010352,75004400010352,535581.00,53558100.00,0.00,67.02,67.02,26.18,10.23,14019013.53,28.10,...,1,153956.09,0.29,Cook_Inlet,75004400010352,42064400.00,11493700.00,Cook_Inlet,21.46,75004400010352
Cook_Inlet_75004400007996,75004400007996,2825727.00,282572700.00,0.00,65.51,65.51,22.75,9.80,64285852.09,23.95,...,1,67318.94,0.02,Cook_Inlet,75004400007996,208771000.00,73801700.00,Cook_Inlet,26.12,75004400007996
Cook_Inlet_75004400004850,75004400004850,1563074.00,156307400.00,0.00,67.16,67.16,18.79,10.87,29374364.09,18.87,...,NaN,NaN,NaN,NaN,NaN,117098200.00,39209200.00,Cook_Inlet,25.08,75004400004850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75004400010337,75004400010337,344724.00,34472400.00,0.00,35.96,35.96,4.01,4.34,1380818.20,3.03,...,NaN,NaN,NaN,NaN,NaN,28343000.00,6129400.00,Cook_Inlet,17.78,75004400010337
Cook_Inlet_75004400008879,75004400008879,233265.00,23326500.00,0.00,30.82,30.82,1.95,2.25,455340.88,1.92,...,NaN,NaN,NaN,NaN,NaN,19775200.00,3551300.00,Cook_Inlet,15.22,75004400008879
Cook_Inlet_75004400004457,75004400004457,1026297.00,102629700.00,0.00,59.70,59.70,16.52,10.14,16950039.30,14.77,...,NaN,NaN,NaN,NaN,NaN,72416200.00,30213500.00,Cook_Inlet,29.44,75004400004457


In [51]:
df_final.drop(columns=drop_cols, axis = 1, inplace=True)
df_final

,cat_ID_txt_x,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,wtd_slope_PCT90,region_x,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per,wtd_north_per
cat_ID_con,,,,,,,,,,,,,,,,,
Cook_Inlet_75004400005344,75004400005344,450061.00,45006100.00,0.00,70.87,70.87,25.38,13.03,11421329.52,26.37,41.61,Cook_Inlet,202569.36,0.45,1588148.84,3.53,15.46
Cook_Inlet_75004400011755,75004400011755,1055899.00,105589900.00,0.00,76.85,76.85,24.02,15.07,25364061.16,24.16,44.20,Cook_Inlet,5262557.61,4.98,47685961.13,45.16,26.59
Cook_Inlet_75004400010352,75004400010352,535581.00,53558100.00,0.00,67.02,67.02,26.18,10.23,14019013.53,28.10,37.79,Cook_Inlet,62396.23,0.12,153956.09,0.29,21.46
Cook_Inlet_75004400007996,75004400007996,2825727.00,282572700.00,0.00,65.51,65.51,22.75,9.80,64285852.09,23.95,34.55,Cook_Inlet,201660.27,0.07,67318.94,0.02,26.12
Cook_Inlet_75004400004850,75004400004850,1563074.00,156307400.00,0.00,67.16,67.16,18.79,10.87,29374364.09,18.87,33.13,Cook_Inlet,163496.46,0.10,NaN,NaN,25.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75004400010337,75004400010337,344724.00,34472400.00,0.00,35.96,35.96,4.01,4.34,1380818.20,3.03,9.43,Cook_Inlet,8008947.94,23.23,NaN,NaN,17.78
Cook_Inlet_75004400008879,75004400008879,233265.00,23326500.00,0.00,30.82,30.82,1.95,2.25,455340.88,1.92,3.44,Cook_Inlet,213751.02,0.92,NaN,NaN,15.22
Cook_Inlet_75004400004457,75004400004457,1026297.00,102629700.00,0.00,59.70,59.70,16.52,10.14,16950039.30,14.77,31.31,Cook_Inlet,77577.74,0.08,NaN,NaN,29.44


In [52]:
#rename columns
df_final.rename({'cat_ID_txt_x':'cat_ID_txt','region_x':'region'},axis=1, inplace=True)
#Recalculate cat_ID
df_final['cat_ID'] = df_final['cat_ID_txt'].astype(np.float64)
df_final.index.is_unique
# reorder cols
df_final = df_final.reindex(columns=final_cols)
df_final

,cat_ID_txt,wtd_slope_COUNT,wtd_slope_AREA,wtd_slope_MIN,wtd_slope_MAX,wtd_slope_RANGE,wtd_slope_MEAN,wtd_slope_STD,wtd_slope_SUM,wtd_slope_MEDIAN,wtd_slope_PCT90,wtd_lake_area_sqm,wtd_lake_per,wtd_glacier_area_sqm,wtd_glacier_per,wtd_north_per
cat_ID_con,,,,,,,,,,,,,,,,
Cook_Inlet_75004400005344,75004400005344,450061.00,45006100.00,0.00,70.87,70.87,25.38,13.03,11421329.52,26.37,41.61,202569.36,0.45,1588148.84,3.53,15.46
Cook_Inlet_75004400011755,75004400011755,1055899.00,105589900.00,0.00,76.85,76.85,24.02,15.07,25364061.16,24.16,44.20,5262557.61,4.98,47685961.13,45.16,26.59
Cook_Inlet_75004400010352,75004400010352,535581.00,53558100.00,0.00,67.02,67.02,26.18,10.23,14019013.53,28.10,37.79,62396.23,0.12,153956.09,0.29,21.46
Cook_Inlet_75004400007996,75004400007996,2825727.00,282572700.00,0.00,65.51,65.51,22.75,9.80,64285852.09,23.95,34.55,201660.27,0.07,67318.94,0.02,26.12
Cook_Inlet_75004400004850,75004400004850,1563074.00,156307400.00,0.00,67.16,67.16,18.79,10.87,29374364.09,18.87,33.13,163496.46,0.10,NaN,NaN,25.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cook_Inlet_75004400010337,75004400010337,344724.00,34472400.00,0.00,35.96,35.96,4.01,4.34,1380818.20,3.03,9.43,8008947.94,23.23,NaN,NaN,17.78
Cook_Inlet_75004400008879,75004400008879,233265.00,23326500.00,0.00,30.82,30.82,1.95,2.25,455340.88,1.92,3.44,213751.02,0.92,NaN,NaN,15.22
Cook_Inlet_75004400004457,75004400004457,1026297.00,102629700.00,0.00,59.70,59.70,16.52,10.14,16950039.30,14.77,31.31,77577.74,0.08,NaN,NaN,29.44


In [53]:
# Export merged dataframe to csv
huccov_csv_out = os.path.join(outdir,'AKSSF_NHDPlus_awcHuc12_19020302_Covariates.csv')
df_final.to_csv(huccov_csv_out, encoding = 'utf-8')
print('Export all covariates dataframe to csv complete')


Export all covariates dataframe to csv complete
